## Conditional Random Field Model (Grid Search)

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#Custom models
from prepro import readfile, get_sentence, is_number, extract_words,get_label,partial_tags

#Model
from sklearn_crfsuite import CRF

#Evalulation
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn_crfsuite.metrics import flat_f1_score

In [3]:
#import data from my github repo
test = readfile("test.txt")
train =readfile("train.txt")

# filenames = ['train.txt', 'valid.txt']
# with open('Combined.txt', 'w') as outfile:
#     for fname in filenames:
#         with open(fname) as infile:
#             for line in infile:
#                 outfile.write(line)
                
# train =readfile("Combined.txt")

## Orthographic Feature Engineering

In [4]:
#Special Features:
def count_vowel(word):
    '''
    Function returns the number of vowels in token
    '''
    return sum(list(map(word.lower().count, "aeiou")))

def dash(word):
    '''
    The Function returns whether or not the token contains a dash
    '''
    return 1 if "-" in word else 0

def count_consonants(word):
    '''
    The Function returns the number of consonants in a token
    '''
    vowels="aeiou"
    return sum(i not in vowels for i in word)

In [5]:
def word2features(sentence,i):
    word = sentence[i]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'Prefix_2': word[:2],
        'Prefix_3': word[:3], 
        'word.isupper()': word.isupper(),
        'word.islower()': word.islower(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'word.isalpha()': word.isalpha(),
        'word.isalnum()': word.isalnum(), 
        "vowels": count_vowel(word),
        "dash": dash(word),
        "consonants": count_consonants(word),
        'Suffix_2': word[-2:],
        'Suffix_3': word[-3:],
        
    }
    if i > 0:
        '''
        Get the token before in the sentence
        '''
        word1 = sentence[i-1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word Prefix_2': word1[:2],
            '-1:word Prefix_3': word1[:3], 
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:word.islower()': word1.lower(),
            '-1:word.isdigit()': word1.isdigit(),
            '-1:word.isalpha()': word1.isalpha(),
            '-1:word.isalnum()': word1.isalnum(), 
            "-1:word.vowels": count_vowel(word1),
            "-1:word.dash": dash(word1),
            "-1:word.consonants": count_consonants(word1),
            '-1:word.Suffix_2': word1[-2:],
            '-1:word.Suffix_3': word1[-3:],
        })
    else:
        features['START'] = True
        
    if i < len(sentence)-1:
        '''
        Get the token after in the sentence
        '''
        word1 = sentence[i+1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.Prefix_2': word1[:2],
            '+1:word.Prefix_3': word1[:3], 
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:word.islower()': word1.lower(),
            '+1:word.isdigit()': word1.isdigit(),
            '+1:word.isalpha()': word1.isalpha(),
            '+1:word.isalnum()': word1.isalnum(), 
            "+1:word.vowels": count_vowel(word1),
            "+1:word.dash": dash(word1),
            "+1:word.consonants": count_consonants(word1),
            '+1:word.Suffix_2': word1[-2:],
            '+1:word.Suffix_3': word1[-3:],
        })
    else:
        features['END'] = True
    
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def get_all_sentences(dataset):
    sentences=[]
    for i in range(len(dataset)):
        sentences.append(get_sentence(dataset,i+1))
    return sentences

def get_all_labels(dataset):
    labels=[]
    for i in range(len(dataset)):
        #labels.append(partial_tags(get_label(dataset,i+1)))
        labels.append(get_label(dataset,i+1))
    return labels

In [6]:
#get the first sentence
sent = get_sentence(train,1)
#label = partial_tags(get_label(train,1))
label = get_label(train,1)

#print out first sentence
print(sent)
print(label)

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
['I-ORG', 'O', 'I-MISC', 'O', 'O', 'O', 'I-MISC', 'O', 'O']


In [7]:
#Applt feature engineering
train_sents = get_all_sentences(train)
train_labels = get_all_labels(train)
test_sents = get_all_sentences(test)
test_labels = get_all_labels(test)

X_train = [sent2features(s) for s in train_sents]
y_train = train_labels
X_test = [sent2features(s) for s in test_sents]
y_test = test_labels


sub_labels=list(set([item for sublist in train_labels for item in sublist]))
sub_labels.remove("O")


## Grid Search of Full Dataset 

In [10]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

crf_grid = CRF(algorithm='lbfgs',
          max_iterations=50,
          all_possible_transitions=False)

parameters = {'c1':[0.1,0.3],
             'c2':[0.2,0.0]}

f1_scorer = make_scorer(flat_f1_score,
                        average='micro',
                        labels=sub_labels)

grid = GridSearchCV(crf_grid, 
                    parameters,
                    #n_jobs=-1,
                    scoring =f1_scorer,
                    verbose=2,
                    return_train_score=True,
                    cv=3)

In [12]:
%%time
grid_results=grid.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] c1=0.1, c2=0.2 ..................................................
[CV] ................................... c1=0.1, c2=0.2, total=  16.5s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.1s remaining:    0.0s


[CV] c1=0.1, c2=0.2 ..................................................
[CV] ................................... c1=0.1, c2=0.2, total=  16.3s
[CV] c1=0.1, c2=0.2 ..................................................
[CV] ................................... c1=0.1, c2=0.2, total=  14.6s
[CV] c1=0.1, c2=0.0 ..................................................
[CV] ................................... c1=0.1, c2=0.0, total=  16.4s
[CV] c1=0.1, c2=0.0 ..................................................
[CV] ................................... c1=0.1, c2=0.0, total=  16.1s
[CV] c1=0.1, c2=0.0 ..................................................
[CV] ................................... c1=0.1, c2=0.0, total=  14.1s
[CV] c1=0.3, c2=0.2 ..................................................
[CV] ................................... c1=0.3, c2=0.2, total=  15.6s
[CV] c1=0.3, c2=0.2 ..................................................
[CV] ................................... c1=0.3, c2=0.2, total=  15.8s
[CV] c

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  3.6min finished


Wall time: 3min 58s


In [13]:
print('best params:', grid_results.best_params_)
print('best CV score:',grid_results.best_score_)

means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

best params: {'c1': 0.1, 'c2': 0.0}
best CV score: 0.8642938170486253
0.863527 (0.015570) with: {'c1': 0.1, 'c2': 0.2}
0.864294 (0.012664) with: {'c1': 0.1, 'c2': 0.0}
0.859081 (0.014579) with: {'c1': 0.3, 'c2': 0.2}
0.860446 (0.013810) with: {'c1': 0.3, 'c2': 0.0}


# Train and Predict on Test dataset

In [17]:
%%time
crf4 = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.0,
          max_iterations=50,
          all_possible_transitions=False)

#training model
crf4.fit(X=X_train, y=y_train)

#TRAIN SET
pred = crf4.predict(X_train)
#generate report on entire model
report = flat_classification_report(y_pred=pred, y_true=y_train,labels=sub_labels)
print(report)
score=flat_f1_score(y_pred=pred, y_true=y_train,average='micro',labels=sub_labels)
print("Traning f1 Score =",score)


#TEST SET
pred = crf4.predict(X_test)
#generate report on entire model
report = flat_classification_report(y_pred=pred, y_true=y_test,labels=sub_labels)
print(report)
score=flat_f1_score(y_pred=pred, y_true=y_test,average='micro',labels=sub_labels)
print("Test f1 Score =",score)

              precision    recall  f1-score   support

       I-LOC       0.99      1.00      1.00      8286
       B-LOC       1.00      1.00      1.00        11
      I-MISC       0.99      1.00      0.99      4556
       I-PER       1.00      1.00      1.00     11128
       B-ORG       1.00      1.00      1.00        24
      B-MISC       0.96      0.73      0.83        37
       I-ORG       1.00      0.99      0.99     10001

   micro avg       1.00      1.00      1.00     34043
   macro avg       0.99      0.96      0.97     34043
weighted avg       1.00      1.00      1.00     34043

Traning f1 Score = 0.9961362734871969


C:\Users\dhoperoberts\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

       I-LOC       0.83      0.87      0.85      1919
       B-LOC       0.00      0.00      0.00         6
      I-MISC       0.78      0.77      0.78       909
       I-PER       0.88      0.87      0.88      2773
       B-ORG       0.00      0.00      0.00         5
      B-MISC       0.00      0.00      0.00         9
       I-ORG       0.81      0.77      0.79      2491

   micro avg       0.84      0.83      0.83      8112
   macro avg       0.47      0.47      0.47      8112
weighted avg       0.83      0.83      0.83      8112

Test f1 Score = 0.8323112184847922
Wall time: 28.4 s
